In [1]:
import nltk
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf

import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, SimpleRNN, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter

from keras.utils import to_categorical
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop


In [2]:
#Downloading nltk StopWords and Wordnet 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
dsk = pd.read_excel('/kaggle/input/kurdishenglish/KDFND_Anlyzed_Cleaned_Filtered_Labeld.xlsx')


In [5]:
# Remove null Rows
dsk = dsk.dropna(subset=['Text_Translate_to_English'])
dsk = dsk.dropna(subset=['Text'])

# Remove columns that not needed
dsk["Article"] = dsk["Text_Translate_to_English"]
dsk['label'] = dsk['label'].map({'Real': 0, 'Fake': 1})  # Convert labels to 0 and 1
dsk = dsk[['Article', 'label']].dropna()

In [6]:
dsk.isna().sum()

Article    0
label      0
dtype: int64

In [7]:
# Check for null/empty rows
print(dsk['Article'].isna().sum())  # Count NaN entries
print((dsk['Article'].str.strip() == '').sum())  # Count empty strings

# Remove rows where 'Article' is an empty string after stripping whitespace
dsk = dsk[dsk['Article'].str.strip() != '']

# Check for null/empty rows
print(dsk['Article'].isna().sum())  # Count NaN entries
print((dsk['Article'].str.strip() == '').sum())  # Count empty strings

0
0
0
0


In [8]:
# Assuming dsk is your DataFrame
# Separate the majority and minority classes
df_majority = dsk[dsk['label'] == 0]
df_minority = dsk[dsk['label'] == 1]

# Oversample the minority class
#df_minority_oversampled = df_minority.sample(len(df_majority), replace=True, random_state=2020)
df_majority_undersampled = df_majority.sample(len(df_minority), replace=True, random_state=2020)

# Combine the majority class with the oversampled minority class
#dsk_balanced = pd.concat([df_majority, df_minority_oversampled])
dsk_balanced = pd.concat([df_minority, df_majority_undersampled])

# Shuffle the dataset
#dskb = dsk_balanced.sample(frac=1, random_state=2020).reset_index(drop=True)

# Check the new class distribution
#grouped_by_class_balanced = dskb.groupby('label').count()
#print(grouped_by_class_balanced)

In [9]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
dskb=dsk
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(dskb['label']), y=dskb['label'])
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class weights:", class_weights_dict)

Class weights: {0: 0.994670055762448, 1: 1.0053873730332603}


In [10]:
def wordpre(text):
    if not isinstance(text, str):
        return ""  # Return empty string for non-string inputs
    text = text.lower()
    # Remove URLs, special characters, and numbers
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove numbers
    
    # Remove extra spaces
    text = text.strip()
    
    return text
    
##  Applying the wordpre method to the dataset
dskb['Article']=dskb['Article'].apply(wordpre)

In [11]:
from collections import Counter
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
punctuation = list(string.punctuation)

stop_words.update(punctuation)

# Apply the function to each article
dskb['Article'] = dskb['Article'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words ) )

dskb.head()


,Article,label
0,archive citizens burn iraqi passports together...,1
1,still beautiful put heart,1
2,ranya hospital crowded according information f...,1
3,ranya,1
4,good friends care close friends understand tru...,1


In [12]:

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dskb['Article'])
word_index = tokenizer.word_index
print(f"Vocabulary size: {len(word_index)}")


Vocabulary size: 41194


In [13]:
glove_twitter = "/kaggle/input/glove-twitter/glove.twitter.27B.200d.txt"

In [14]:
# Load GloVe vectors
embedding_index = {}
with open(glove_twitter, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vectors

print(f"Loaded {len(embedding_index)} word vectors.")

Loaded 1193514 word vectors.


In [15]:

# Create embedding matrix
embedding_dim = 200
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (41195, 200)


In [16]:
# Example GloVe embedding matrix
glove_embedding_matrix = np.random.rand(len(word_index) + 1, 200)  # Random example

# L2 normalization of GloVe embeddings
l2_norm = np.linalg.norm(glove_embedding_matrix, axis=1, keepdims=True)

# Check for zero vectors and handle them
# Set zero vectors to a small value to avoid division by zero
l2_norm[l2_norm == 0] = 1e-10  # or any small constant

# Normalize the embeddings
normalized_glove_embeddings = embedding_matrix / l2_norm

# Verify the shape of the normalized embeddings
print("Normalized GloVe Embeddings Shape:", normalized_glove_embeddings.shape)

print("l2_norm:", l2_norm)

Normalized GloVe Embeddings Shape: (41195, 200)
l2_norm: [[8.40894605]
 [8.32787691]
 [8.65937445]
 ...
 [8.02604343]
 [8.54577345]
 [8.47406998]]


In [17]:

# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
#X = dskb['Article']
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
#model.add(Input(shape=()))
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=100, # Pad sequences to this length
                    trainable=True)) # Freeze embedding layer
model.add(LSTM(70,return_sequences=True, input_shape=(X_train.shape[1], 1)))   # 3D input
model.add(Dropout(0.2))
model.add(Conv1D(filters=100, kernel_size=2, activation='relu'))
model.add(GlobalMaxPooling1D())  # Global Max Pooling
model.add(Dense(260, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │      36,860,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_6               │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,860,000 (140.61 MB)

 Trainable params: 36,860,000 (140.61 MB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/5
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 644s 508ms/step - accuracy: 0.7646 - loss: 0.4868 - val_accuracy: 0.8681 - val_loss: 0.3239 - learning_rate: 0.0010
Epoch 2/5
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 635s 506ms/step - accuracy: 0.9199 - loss: 0.2099 - val_accuracy: 0.8749 - val_loss: 0.3111 - learning_rate: 0.0010
Epoch 3/5
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 635s 505ms/step - accuracy: 0.9630 - loss: 0.1021 - val_accuracy: 0.8683 - val_loss: 0.3535 - learning_rate: 0.0010
Epoch 4/5
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 634s 505ms/step - accuracy: 0.9792 - loss: 0.0589 - val_accuracy: 0.8712 - val_loss: 0.4609 - learning_rate: 0.0010
Epoch 5/5
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 624s 497ms/step - accuracy: 0.9868 - loss: 0.0393 - val_accuracy: 0.8669 - val_loss: 0.5278 - learning_rate: 0.0010
628/628 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.8728 - loss: 0.3166
Validation Accuracy: 0.8749253153800964


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=100, # Pad sequences to this length
                    trainable=False)) # Freeze embedding layer
model.add(LSTM(128, return_sequences=False, input_shape=(X_train.shape[1], 1)))  # 3D input
model.add(Dropout(0.4))
#model.add(Bidirectional(LSTM(70)))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │       8,239,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,239,000 (31.43 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 8,239,000 (31.43 MB)

None


In [19]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 176s 138ms/step - accuracy: 0.7269 - loss: 0.5480 - val_accuracy: 0.7766 - val_loss: 0.4815 - learning_rate: 0.0010
Epoch 2/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 173s 137ms/step - accuracy: 0.7818 - loss: 0.4781 - val_accuracy: 0.7880 - val_loss: 0.4608 - learning_rate: 0.0010
Epoch 3/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 173s 137ms/step - accuracy: 0.7990 - loss: 0.4474 - val_accuracy: 0.7963 - val_loss: 0.4481 - learning_rate: 0.0010
Epoch 4/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 174s 138ms/step - accuracy: 0.8119 - loss: 0.4222 - val_accuracy: 0.7992 - val_loss: 0.4423 - learning_rate: 0.0010
Epoch 5/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 173s 137ms/step - accuracy: 0.8145 - loss: 0.4172 - val_accuracy: 0.7981 - val_loss: 0.4429 - learning_rate: 0.0010
Epoch 6/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 174s 138ms/step - accuracy: 0.8262 - loss: 0.3964 - val_accuracy: 0.8019 - val_loss: 0.4452 - learning_rate: 0.0010
Epoch 7/10
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 175s 138ms

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
#model.add(Input(shape=()))
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    weights=[normalized_glove_embeddings],
                    input_length=100, # Pad sequences to this length
                    trainable=False)) # Freeze embedding layer
model.add(LSTM(128, return_sequences=False, input_shape=(X_train.shape[1], 1)))  # 3D input
#model.add(Dropout(0.2))
#model.add(LSTM(64))
model.add(Dropout(0.4)),
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │       7,875,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,875,600 (30.04 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,875,600 (30.04 MB)

None


In [19]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 165s 129ms/step - accuracy: 0.7925 - loss: 0.4437 - val_accuracy: 0.8825 - val_loss: 0.2635 - learning_rate: 0.0010
Epoch 2/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 164s 130ms/step - accuracy: 0.8877 - loss: 0.2569 - val_accuracy: 0.8911 - val_loss: 0.2438 - learning_rate: 0.0010
Epoch 3/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 165s 132ms/step - accuracy: 0.8936 - loss: 0.2412 - val_accuracy: 0.8971 - val_loss: 0.2299 - learning_rate: 0.0010
Epoch 4/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 165s 132ms/step - accuracy: 0.8969 - loss: 0.2303 - val_accuracy: 0.7825 - val_loss: 0.5020 - learning_rate: 0.0010
Epoch 5/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 165s 131ms/step - accuracy: 0.7495 - loss: 0.5088 - val_accuracy: 0.8824 - val_loss: 0.2724 - learning_rate: 0.0010
Epoch 6/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 165s 132ms/step - accuracy: 0.8872 - loss: 0.2627 - val_accuracy: 0.8920 - val_loss: 0.2361 - learning_rate: 0.0010
Epoch 7/10
1256/1256 ━━━━━━━━━━━━━━━━━━━━ 166s 132ms

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, MaxPooling1D, SimpleRNN, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization,SpatialDropout1D
from keras.optimizers import RMSprop

# Define model
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Embedding(input_dim=len(word_index) + 1,
                   output_dim=embedding_dim,weights=[embedding_matrix],input_length=100,trainable=False)) # Freeze embedding layer
model.add(LSTM(166, return_sequences=False))
model.add(Dropout(0.3)),
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=1e-3), metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 100, 200)            │       7,875,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 166)                 │         243,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 166)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │           2,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,121,977 (30.98 MB)

 Trainable params: 246,377 (962.41 KB)

 Non-trainable params: 7,875,600 (30.04 MB)

None


In [23]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 301s 119ms/step - accuracy: 0.8529 - loss: 0.3285 - val_accuracy: 0.8986 - val_loss: 0.2220 - learning_rate: 0.0010
Epoch 2/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 297s 118ms/step - accuracy: 0.9096 - loss: 0.2062 - val_accuracy: 0.9119 - val_loss: 0.2000 - learning_rate: 0.0010
Epoch 3/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 302s 120ms/step - accuracy: 0.9240 - loss: 0.1797 - val_accuracy: 0.9170 - val_loss: 0.1911 - learning_rate: 0.0010
Epoch 4/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 298s 119ms/step - accuracy: 0.9364 - loss: 0.1572 - val_accuracy: 0.9215 - val_loss: 0.1940 - learning_rate: 0.0010
Epoch 5/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 298s 119ms/step - accuracy: 0.9504 - loss: 0.1257 - val_accuracy: 0.9286 - val_loss: 0.1934 - learning_rate: 0.0010
Epoch 6/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 299s 119ms/step - accuracy: 0.9617 - loss: 0.1002 - val_accuracy: 0.9291 - val_loss: 0.1990 - learning_rate: 0.0010
Epoch 7/10
2511/2511 ━━━━━━━━━━━━━━━━━━━━ 299s 119ms

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, History
import tensorflow as tf
from tensorflow.keras.regularizers import l2

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(dskb['Article'], dskb['label'], test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)  # Encode training labels
y_test = label_encoder.transform(y_test)  # Encode test labels

# Text preprocessing
max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                   output_dim=embedding_dim,weights=[embedding_matrix],input_length=max_len,trainable=False))
model.add(LSTM(166, dropout=0.4, recurrent_dropout=0.38, kernel_regularizer=l2(0.01)))
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = History()
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[history, early_stopping])

# Predict
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype("int32")

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1005/1005 ━━━━━━━━━━━━━━━━━━━━ 175s 172ms/step - accuracy: 0.7539 - loss: 0.9195 - val_accuracy: 0.4948 - val_loss: 0.7790
Epoch 2/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 171s 170ms/step - accuracy: 0.8037 - loss: 0.4917 - val_accuracy: 0.8808 - val_loss: 0.3583
Epoch 3/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 172s 171ms/step - accuracy: 0.8481 - loss: 0.4200 - val_accuracy: 0.6084 - val_loss: 0.7134
Epoch 4/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 171s 170ms/step - accuracy: 0.6307 - loss: 0.6837 - val_accuracy: 0.6367 - val_loss: 0.9115
Epoch 5/5
1005/1005 ━━━━━━━━━━━━━━━━━━━━ 173s 172ms/step - accuracy: 0.6173 - loss: 0.8280 - val_accuracy: 0.7315 - val_loss: 0.6455
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 2.
628/628 ━━━━━━━━━━━━━━━━━━━━ 32s 51ms/step
Accuracy: 0.8826487428429176
Precision: 0.8664734299516909
Recall: 0.9019410640651715
F1 Score: 0.883851574434534


In [27]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train model
history = model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_split=0.2, callbacks=[history, early_stopping])

# Evaluate model
val_loss, val_accuracy = model.evaluate(X_test_pad, y_test)
print(f"Validation Accuracy: {val_accuracy}")

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 64269
'y' sizes: 56235


In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)


In [19]:
import optuna
import numpy as np
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#tokenizer = Tokenizer()
# Pad sequences
X = pad_sequences(tokenizer.texts_to_sequences(dskb['Article']), maxlen=100)
y = dskb['label']

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Ensure labels are NumPy arrays and of the correct type
y_train = np.array(y_train, dtype=np.float32)
y_val = np.array(y_val, dtype=np.float32)

embedding_dim = 100  # Fixed dimension matching your embedding matrix

# Define the objective function for Optuna
def create_model(trial):
    model = Sequential()
    
    # Hyperparameters to optimize
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 200])
    conv_filters = trial.suggest_int('conv_filters', 32, 128)
    kernel_size = trial.suggest_int('kernel_size', 2, 5)
    lstm_units = trial.suggest_int('lstm_units', 50, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    lstm_units = trial.suggest_int('lstm_units', 50, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    # Dynamically adjust embedding_matrix
    adjusted_embedding_matrix = embedding_matrix[:, :embedding_dim]
    
    
    model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=embedding_dim,
                    #weights=[adjusted_embedding_matrix],  # Use the pretrained weights
                    trainable=False))  # Trainable embeddings
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.3)),
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def objective(trial):
    model = create_model(trial)
    
    # Train the model
    history = model.fit(X_train, y_train,
                        epochs=3,
                        batch_size=64,
                        validation_data=(X_val, y_val),
                        verbose=0)
    # Perform garbage collection
    gc.collect()
    
    # Get the validation accuracy
    val_accuracy = history.history['val_accuracy'][-1]
    return val_accuracy

# Create a study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)
# Run Optuna study
#study = optuna.create_study(direction="minimize")  # Minimize (1 - accuracy) to maximize accuracy

# Best hyperparameters and accuracy
print("Best hyperparameters:", study.best_params)
print("Best trial validation accuracy:", 1 - study.best_value)
# Print validation accuracy of the best trial
best_trial = study.best_trial
print("Best validation accuracy:", best_trial.value)


[I 2025-01-11 10:55:46,391] A new study created in memory with name: no-name-d82aa61a-b38d-4a40-9b48-9a07a52ac02b
[I 2025-01-11 10:57:52,600] Trial 0 finished with value: 0.706632137298584 and parameters: {'embedding_dim': 100, 'conv_filters': 128, 'kernel_size': 5, 'lstm_units': 253, 'dropout_rate': 0.30567700937688125}. Best is trial 0 with value: 0.706632137298584.
[I 2025-01-11 10:59:59,396] Trial 1 finished with value: 0.6993626952171326 and parameters: {'embedding_dim': 50, 'conv_filters': 90, 'kernel_size': 5, 'lstm_units': 176, 'dropout_rate': 0.43471167156770185}. Best is trial 0 with value: 0.706632137298584.
[I 2025-01-11 11:02:06,212] Trial 2 finished with value: 0.7129058241844177 and parameters: {'embedding_dim': 100, 'conv_filters': 71, 'kernel_size': 4, 'lstm_units': 168, 'dropout_rate': 0.3085457006568489}. Best is trial 2 with value: 0.7129058241844177.
[I 2025-01-11 11:04:13,034] Trial 3 finished with value: 0.7074288129806519 and parameters: {'embedding_dim': 100, '

Best hyperparameters: {'embedding_dim': 200, 'conv_filters': 39, 'kernel_size': 4, 'lstm_units': 199, 'dropout_rate': 0.45368274304766243}
Best trial validation accuracy: 0.2864469289779663
Best validation accuracy: 0.7135530710220337


In [ ]:
import optuna.visualization as vis


import matplotlib.pyplot as plt

# Plot optimization history (accuracy over trials)
optuna.visualization.plot_optimization_history(study)
#plt.show()

# Plot parameter importances (which hyperparameters affected accuracy most)
optuna.visualization.plot_param_importances(study)
#plt.show()
